# 异步网络编程的组件介绍


asyncio提供了一系列的工具用于异步网络编程.由于各种原因,它的很多接口并不能跨平台.对于类Unix系统asyncio是完全支持的,但对于windows就不一定了,这点需要注意.

其中主要的部件有:


+ 建立连接和服务器的工具
+ Transports(通信)和protocols(协议)类
+ 读写流

底层的接口有

+ Socket中的常量
+ asyncio中事件循环的socket底层api

其他相关的工具包括

+ 信号量和信号处理接口
+ Future对象操作
+ 异步异常处理工具

## 主要的异步网络编程部件

这一部分是最主要的网络编程组件,一般情况下使用这部分的组件就已经可以进行网络编程了

### 建立连接和服务器的工具

套接字(socket)也是有几种的,常用的包括

+ TCP套接字,最常见的套接字
+ UDP套接字,只发送不验证的套接字,可以广播
+ unix套接字,用于类Unix系统上同一主机间不同进程通信

#### 建立连接

+ `coroutine AbstractEventLoop.create_connection(protocol_factory, host=None, port=None, *, ssl=None, family=0, proto=0, flags=0, sock=None, local_addr=None, server_hostname=None)->(transport, protocol)`
    
    事件循环对象中用于创建TCP连接的协程,family `AF_INET` 或者`AF_INET6`,socket类型`SOCK_STREAM`.
    + `host`和`port`
        
        用于指定要连接的对象,会自己解析ip地址来判断使用的v4协议(`AF_INET`)还是v6协议(`AF_INET6`),也可以通过`family`参数进行指定其他可以设置的包括`proto`, `flags`都是用于设置连接的参数
        
    + `protocol_factory`
    
        参数要求是一个类或者工厂函数,用于生成`protocols`对象
        
    + `ssl`
    
        指定使用的ssl证书,可以配合`server_hostname`参数指定证书的服务器地址
        
    + `sock`
    
        用于指定套接字,使用这个参数必须`host`,`port`都为`None`
            
    + `local_addr`
    
        用于指定本地用于连接远端的端口和服务器地址,不设置的话表示自动生成
        
        
    这个协程一旦执行完成其结果为一个由transport和 protocol实例组成的对,通常用于写客户端
    
    
+ `coroutine asyncio.open_connection(host=None, port=None, *, loop=None, limit=None, **kwds)->(StreamReader,StreamWriter)`

    这个协程是上面`AbstractEventLoop.create_connection`的封装,会建立一个指向指定主机端口的TCP连接,一旦建立成功会返回一组(StreamReader,StreamWriter)对用于交互,而所有操作都是使用返回的两个对象StreamReader,StreamWriter来实现的.这个接口不使用`Transports(通信)和protocols(协议)类`而是使用其创建出来的读写对象来实现,相对而言更加适合用于写客户端


+ `coroutine AbstractEventLoop.create_unix_connection(protocol_factory, path, *, ssl=None, sock=None, server_hostname=None)->(transport, protocol)`

    这个接口用于创建Unix连接,用于同一主机上多进程间通信.socket family `AF_UNIX, socket`,类型`SOCK_STREAM`,这个接口只在Unix系统上有用
    
    + `path`是Unix套接字域名,除非`sock`参数被指定否咋就没有必要.这个接口比较不常用,本文不做阐述
    
    
+ `coroutine asyncio.open_unix_connection(path=None, *, loop=None, limit=None, **kwds)->(StreamReader,StreamWriter)`

    + `create_unix_connection()`接口的封装,同样只能用在Unix上.



+ `coroutine AbstractEventLoop.create_datagram_endpoint(protocol_factory, local_addr=None, remote_addr=None, *, family=0, proto=0, flags=0, reuse_address=None, reuse_port=None, allow_broadcast=None, sock=None)->(transport, protocol)`

    与上面相似,但是是用于建立UDP连接的.socket family` AF_INET`或者`AF_INET6`,socket 类型为`SOCK_DGRA`.和TCP协议不同,使用UDP协议进行信息的传输之前不需要建议连接.换句话说就是客户端向服务器发送信息,客户端只需要给出服务器的ip地址和端口号即可,然后将信息封装到一个待发送的报文中并且发送出去.至于远端是否存在,本地根本不管.UDP协议更多的是用在p2p通信上
    
    UDP因为不用建立连接,所以可以广播,消息发送到一个多播地址,那么所有该地址下的主机都会被发送一份这个要广播的信息.
    
    + `protocol_factory`
    
        参数要求是一个类或者工厂函数,用于生成`protocols`对象
        
    + `local_addr`
    
        参数为(local_host, local_port)的形式,设置本地的地址和端口
    
    + `remote_addr`
    
        参数为(remote_host, remote_port)的形式,设置本地的地址和端口
        
    + `family, proto, flags`
    
        和上面一样,由于设置socket.
        
    + `sock`
    
        用于指定套接字,和上面一样.针对UDP的话,如果要让UDP进行多播,那么更好的方式是指定sock对象,比如:
        
        ```python
        def make_sock():
            sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
            sock.bind(('', 1900))
            group = socket.inet_aton('239.255.255.250')
            mreq = struct.pack('4sL', group, socket.INADDR_ANY)
            sock.setsockopt(socket.IPPROTO_IP, socket.IP_ADD_MEMBERSHIP, mreq)
            sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
            sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEPORT, 1)
            return sock
            
        loop.create_datagram_endpoint(my_udp_protocol, sock=make_sock())
        ```

    + `reuse_address` 
    
        内核在time_wait状态下重用本地套接字,而不必等待其自然超时过期.
    
    + `reuse_port` 
    
        告诉内核允许该端点绑定到与其他现有端点绑定到的相同端口,只要它们在创建时都设置了该标志即可.这个选项在windows和一些unix上不支持.如果so_reuseport常量没有定义,那么这个功能是不支持的.
        
    + `allow_broadcast` 
    
        允许这个端点发送广播消息,允许广播的话那么局域网内的所有地址都会被发送一遍.

#### 创建服务器 


+ `coroutine AbstractEventLoop.create_server(protocol_factory, host=None, port=None, *, family=socket.AF_UNSPEC, flags=socket.AI_PASSIVE, sock=None, backlog=100, ssl=None, reuse_address=None, reuse_port=None)->Server`

    创建一个TCP服务器(socket类型为`SOCK_STREAM`),其返回值为一个Server对象,这个对象包含`close`方法用于关闭监听循环,协程`wait_closed()`用于等待剩下的执行完毕然后关闭监听循环,`sockets`参数用于保存已经建立连接的套接字对象.
    
    + `backlog`
    
        系统队列的最大连接数
    
    + `protocol_factory`
    
        为协议对象的工厂函数,
        
    + `host=None, port=None, *, family=socket.AF_UNSPEC, flags=socket.AI_PASSIVE, sock=None, ssl=None, reuse_address=None, reuse_port=None`
    
        用于设定socket对象和ssl的设置,和创建连接的部分一致
    

+ `coroutine asyncio.start_server(client_connected_cb(client_reader, client_writer)->None, host=None, port=None, *, loop=None, limit=None, **kwds)->Server`

    `create_server`的包装,其返回值也是一个Server对象.使用回调函数或者协程函数`client_connected_cb`来规定服务器的行为,其参数为`client_reader, client_writer`,他们分别是`StreamReader`和`StreamWriter`的实例.
    
    `limit`参数用于设置缓冲区的大小限制.

+ `coroutine AbstractEventLoop.create_unix_server(protocol_factory, path=None, *, sock=None, backlog=100, ssl=None)->Server`

    创建一个Unix服务器(socket类型为`AF_UNIX`).参数和前文提到的意义一样.只能用在Unix系统上.

+ `coroutine asyncio.start_unix_server(client_connected_cb, path=None, *, loop=None, limit=None, **kwds)->Server`

    创建一个Unix服务器(socket类型为`AF_UNIX`).参数和前文提到的意义一样.只能用在Unix系统上.

### Transports(通信)和protocols(协议)类

protocols(协议)类用于定义网络服务的行为,